# Tests on Extreme Inputs

The following test section has the purpose is comprising tests that we expect to fail. This comprise tests that will show the limitation when working with floating point numbers. In particular this relates to testing for input values (x) in our matrices that exceed our limit of x < sqrt(2**53/n). Likewise, we test for very small numbers between 0.0-1.0 that include decimals beyond what we expect for floating point accuracy.

#### 1) Test on Large Numbers: x > sqrt(2**53/n)

#### 2) Test on Small Numbers: 0.0 < x < 1.0

All tests are run with: numpy.assert_equals() numpy.assert_allclose() https://numpy.org/doc/stable/reference/generated/numpy.allclose.html


In [1]:
import sys
sys.path.append("/home/gustavgyrst/Desktop/AA_Final/TheMatrix/")
#sys.setrecursionlimit(2000) Probably not necessary
from experiments import *
from matrix_implementations import *
import random
import numpy as np
import csv
import logging

### Correct Input Generator (Following x < np.sqrt(2**(53)/n))

In [27]:
def get_input_range(n):
    lower_bound = 0
    upper_bound = round(np.sqrt(float(2**(53))/n))
    input_range = [lower_bound, upper_bound]
    return input_range


def generate_input(n: int) -> np.ndarray:
    list= []
    input_range = get_input_range(n)
    for i in range(0,n*n):
        l = random.randint(input_range[0],int(input_range[1]))
        list.append(float(l))
    return np.array(list).reshape(n,n)



### Too Large Inputs 


In [3]:
def too_large_input_range(n, power):
    lower_bound = 0
    upper_bound = round(np.sqrt(float(2**(power)/n)))
    input_range = [lower_bound, upper_bound]
    return input_range

def generate_too_large_input(n, power=70):
    list= []
    input_range = too_large_input_range(n, power)
    for i in range(0,n*n):
        
        l = random.randint(input_range[0],int(input_range[1]))
        list.append(float(l))
    return np.array(list).reshape(n,n)


### Test Matrix Generation

In [4]:
n = 64
A = generate_too_large_input(n)
B = generate_too_large_input(n)
C = generate_too_large_input(n)


In [5]:
A_within = generate_input(n)
B_within = generate_input(n)
type(A)

numpy.ndarray

In [6]:
np_res = np.matmul(A,B)
res = elementary_multiplication(Matrix(n,n, A), Matrix(n,n,B))

In [7]:
np_res_within = np.matmul(A_within,B_within)
res_within = elementary_multiplication(Matrix(n,n, A_within), Matrix(n,n,B_within))

In [8]:
#np.testing.assert_allclose(res.tolist(), np_res, 0.01, 0.05)


In [9]:
np.testing.assert_array_equal(res_within.tolist(),np_res_within)

In [10]:
#np.testing.assert_equal(res.tolist(), np_res)


In [37]:
def test(algorithm, A:np.ndarray, B:np.ndarray, n:int, C:Matrix=None):
    try:
        if C == None:
            np.testing.assert_equal(algorithm(Matrix(len(A),len(A), A), Matrix(len(B),len(B),B)).tolist(), np.matmul(A,B))
            print("Ok")
        else:
            np.testing.assert_equal(algorithm(Matrix(len(A),len(A), A), Matrix(len(B),len(B),B), C).tolist(), np.matmul(A,B))
            print("Ok")
    except AssertionError as error:
        
        # Uncomment the log to see full error message each time
        logging.error("TEXT", exc_info=True)
        error_message = str(error)
        error_percentage = float(error_message.split("Mismatched elements: ")[1].split("%)")[0].split("(")[1])/100
        
        return error_percentage

#test(elementary_multiplication, A, B, n)

In [47]:
def full_test(algorithm_to_be_tested, input_generator, n:int, iterations:int, inplace=False, power:int=None):
    
    """full_test takes an algorithm that is to be tested and the specified input generator that creates the inputs for 
    the given test. It evaluates whether a power is specified. Here a power refers to the power to which the input is
    scaled with. it also checks if the algorithm being tested is an inplace algorithm or copying variant.
    
    """
    errors = []
    
    for i in range(iterations):
        
        if power != None:
            A = input_generator(n, power)
            B = input_generator(n, power)
        else: 
            A = input_generator(n)
            B = input_generator(n)
            
        if inplace == True:     
            val = test(algorithm_to_be_tested, A, B, C=Matrix(len(A),len(A)), n=n)
            if val != None:
                errors.append(val)
        else:
            errors  = test(algorithm_to_be_tested, A, B, n=n)
            if val != None:
                errors.append(val)
    
    return errors

In [48]:
full_test(recursive_multiplication_write_through, generate_input, n, iterations=3, inplace=True)

Ok
Ok
Ok


[]